<a href="https://colab.research.google.com/github/MaxMatteucci/mgmt467-analytics-portfolio/blob/main/Lab5_Classification_BQMLBrightspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 🧱 Lab 5: Predicting Flight Diversions with BigQueryML  
**Author:** Max Matteucci  
**Course:** MGMT 467 – Big Data and Cloud Analytics  
**Project ID:** `database-project-467`  
**Dataset:** `flights.flights_classification`  

**Objective:**  
The goal of this lab is to


In [2]:
from google.colab import auth
from google.cloud import bigquery

# Authenticate and initialize BigQuery client
auth.authenticate_user()
client = bigquery.Client(project="database-project-467")
print("✅ Connected to BigQuery project:", client.project)


✅ Connected to BigQuery project: database-project-467


✈️ 3. Business Context

A flight diversion means a plane lands somewhere other than its scheduled destination.
Airlines want to identify flights with a high risk of diversion.

Cost trade-offs:

False Positive: Predicting a diversion that does not happen → wastes crew and logistics effort.

False Negative: Missing a real diversion → leads to operational disruption and angry customers.
Usually, false negatives are more expensive since the airline fails to prepare for actual diversion

In [3]:
prompt = """
# TASK: Generate a BQML query to create a classification model.
# GOAL: Predict the 'diverted' column using 'distance', 'carrier', and 'dep_delay' as features.
# SPECIFICATIONS: The model type must be 'LOGISTIC_REG'.
"""


In [5]:
create_label_query = """
CREATE OR REPLACE TABLE `database-project-467.flights.flights_classification` AS
SELECT
  arr_delay,
  dep_delay,
  distance,
  carrier,
  origin,
  month,
  CASE WHEN arr_delay > 60 THEN TRUE ELSE FALSE END AS diverted
FROM
  `database-project-467.flights.flights`
WHERE
  arr_delay IS NOT NULL;
"""

client.query(create_label_query).result()
print("✅ Created classification table with diverted label.")


✅ Created classification table with diverted label.


In [6]:
create_model_query = """
CREATE OR REPLACE MODEL `database-project-467.flights.flight_diversion_classifier`
OPTIONS(
  model_type='LOGISTIC_REG',
  input_label_cols=['diverted'],
  enable_global_explain=TRUE
) AS
SELECT
  diverted,
  dep_delay,
  distance,
  carrier
FROM
  `database-project-467.flights.flights_classification`;
"""

job = client.query(create_model_query)
job.result()
print("✅ Logistic regression model created successfully!")


✅ Logistic regression model created successfully!


In [7]:
evaluate_query = """
SELECT *
FROM ML.EVALUATE(MODEL `database-project-467.flights.flight_diversion_classifier`);
"""
eval_df = client.query(evaluate_query).to_dataframe()
eval_df


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.917367,0.752009,0.972734,0.826498,0.086067,0.971097


In [8]:
predict_query = """
SELECT *
FROM ML.PREDICT(
  MODEL `database-project-467.flights.flight_diversion_classifier`,
  (
    SELECT
      30 AS dep_delay,
      2000 AS distance,
      'AA' AS carrier
  )
);
"""

pred_df = client.query(predict_query).to_dataframe()
pred_df


,predicted_diverted,predicted_diverted_probs,dep_delay,distance,carrier
0,False,"[{'label': True, 'prob': 0.05592439256958033},...",30,2000,AA


In [9]:
predict_threshold_query = """
SELECT *
FROM ML.PREDICT(
  MODEL `database-project-467.flights.flight_diversion_classifier`,
  (
    SELECT
      30 AS dep_delay,
      2000 AS distance,
      'AA' AS carrier
  ),
  STRUCT(0.75 AS threshold)
);
"""

client.query(predict_threshold_query).to_dataframe()


,predicted_diverted,predicted_diverted_probs,dep_delay,distance,carrier
0,False,"[{'label': True, 'prob': 0.05592439256958033},...",30,2000,AA


### 🧾 **Lab 5 Summary: Predicting Flight Diversions with BigQueryML**

**Objective:**  
This lab used BigQueryML to train and evaluate a **logistic regression** model that predicts whether a flight would be **diverted**. The dataset did not contain a diversion label, so a synthetic `diverted` column was created based on flights with an arrival delay greater than 60 minutes.  

**Key Steps:**  
1. Created a new table with a binary label (`diverted = TRUE` if `arr_delay > 60`).  
2. Trained a **LOGISTIC_REG** model using `dep_delay`, `distance`, and `carrier` as predictors.  
3. Evaluated the model with **precision**, **recall**, and a **confusion matrix** from `ML.EVALUATE`.  
4. Interpreted the results using Gemini, explaining what false positives and false negatives mean in business terms.  
5. Used a **custom threshold of 0.75** in `ML.PREDICT` to explore how airlines can reduce false alarms when predicting diversions.  

**Outcome:**  
The model performed well in identifying non-diverted flights but missed some rare diversions, which is common in imbalanced data. This lab demonstrated how to build, evaluate, and interpret a **classification model** in BigQueryML and how adjusting thresholds changes business decisions for airlines.
